In [3]:
import pandas as pd
import math
import datetime
import matplotlib.pyplot as plt
import random
import seaborn as sns
from sklearn.linear_model import LinearRegression as linreg





In [4]:
cab_data = pd.read_csv("Cab_Data.csv", index_col="Transaction ID")
cab_data


city_data = pd.read_csv("City.csv", index_col="City")
city_data["Population"] = city_data["Population"].str.replace(',', '').astype(int)
city_data["Users"] = city_data["Users"].str.replace(',', '').astype(int)
city_data

#  The population and users columns data are stored as strings 
EY	1177609	27247
ORANGE COUNTY	1030185	12994
SAN DIEGO CA	959307	69995
PHOENIX AZ	943999	6133
DALLAS TX	942908	22157
ATLANTA GA	814885	24701
DENVER CO	754233	12421
AUSTIN TX	698371	14978
SEATTLE WA	671238	25063
TUCSON AZ	631442	5712
SAN FRANCISCO CA	629591	213609
SACRAMENTO CA	545776	7044
PITTSBURGH PA	542085	3643
WASHINGTON DC	418859	127001
NASHVILLE TN	327225	9270

SyntaxError: invalid syntax (1151011071.py, line 11)

In [ ]:
customer_data = pd.read_csv("Customer_ID.csv", index_col="Customer ID")


transaction_data = pd.read_csv("Transaction_ID.csv", index_col="Transaction ID")
transaction_data


######
pd.to_datetime(cab_data["Date of Travel"].min(), unit="d", origin='1899-12-30')
Timestamp('2016-01-02 00:00:00')
pd.to_datetime(cab_data["Date of Travel"].max(), unit="d", origin='1899-12-30')
Timestamp('2018-12-31 00:00:00')
In the next two commands we add two columns:

The first column changes the dates into readable format.
The second column calculates the profit or loss made from each trip.
cab_data["Date of Travel"] = pd.to_datetime(cab_data["Date of Travel"], unit="d", origin='1899-12-30')
cab_data["Profit Generated"] = cab_data["Price Charged"] - cab_data["Cost of Trip"]
Merging the cab_data and transaction_data dataframes 

cab_data = transaction_data.join(cab_data)



In [ ]:
#Profit per trip in different cities
 investigating which cities have a higher (on average) profit generated per trip, and which taxi company generates more profit per trip in each city.

grouped_cab_data = cab_data.groupby(["City", "Company"])["KM Travelled", "Price Charged", "Cost of Trip", "Profit Generated"]

grouped_cab_data_summary = grouped_cab_data.mean()

grouped_cab_data_summary



grouped_cab_data_summary["Profit Generated"].unstack().plot.bar(stacked=True)
<AxesSubplot:xlabel='City'>


In [ ]:
#The top 3 cities in which Yellow Cab is the most profitable are Dallas, New York, and Silicon Valley. For Pink Cab, the top 3 cities are Miami, New York, and Silicon Valley.


cab_data["Trips"] = 1
cab_data["Yellow Cab Trips"] = cab_data["Company"].map(lambda company: 1 if company == "Yellow Cab" else 0)
cab_data["Pink Cab Trips"] = cab_data["Company"].map(lambda company: 1 if company == "Pink Cab" else 0)

grouped_cab_data_counts = cab_data.groupby("City")["Yellow Cab Trips", "Pink Cab Trips", "Trips"].sum()

grouped_cab_data_counts["Share of Yellow Cab Trips"] = grouped_cab_data_counts["Yellow Cab Trips"] / grouped_cab_data_counts["Trips"]
grouped_cab_data_counts["Share of Pink Cab Trips"] = grouped_cab_data_counts["Pink Cab Trips"] / grouped_cab_data_counts["Trips"]
####
grouped_cab_data_counts_merged = grouped_cab_data_counts.join(city_data)
###
merged.plot.scatter("Population", "Share of Yellow Cab Trips")
<AxesSubplot:xlabel='Population', ylabel='Share of Yellow Cab Trips'>

grouped_cab_data_counts_merged.plot.scatter("Users", "Share of Yellow Cab Trips")
<AxesSubplot:xlabel='Users', ylabel='Share of Yellow Cab Trips'>

There seems to be a positive correlation from the data above, meaning that cities with higher population tend to have a higher share of Yellow Cab users. However, to make the data more clear, it may help to transform the population/number of users logarithmically (base 10) to make a better linear fit, then do a linear regression analysis.

grouped_cab_data_counts_merged["log(Population)"] = grouped_cab_data_counts_merged["Population"].map(math.log10)
grouped_cab_data_counts_merged["log(Users)"] = grouped_cab_data_counts_merged["Users"].map(math.log10)



In [ ]:
###########
The results:
share of Yellow Cab trips from the population of the city is:

with the  correlation coefficient of 0.08557, indicating a rather weak relationship.
The model predicting the share of Yellow Cab trips from the number of users in the city is:

with the  correlation coefficient of 0.42026, indicating a fairly strong relationship. This means the share of Yellow Cab trips tends to increase by 17.650% as the number of users in the city increases tenfold.


sns.scatterplot(data=cab_data, x="KM Travelled", y="Profit Generated", hue="Company")
<AxesSubplot:xlabel='KM Travelled', ylabel='Profit Generated'>

While it is clear from visualizing the first two plots that Yellow Cab incurs more costs and charges more than Pink Cab, it is not yet clear what the effect on the profit is. In particular, there are many high outliers for Yellow Cab in terms of profit generated in the trip. One must also note that the data "fans out" for longer trips, i.e. becomes more varied with range approximately proportional to the distance, so it may be worth it to do another visualization, except the y-axis will plot the cost/price/profit per KM Travelled.

cab_data["Profit/KM"] = cab_data["Profit Generated"] / cab_data["KM Travelled"]

sns.scatterplot(data=cab_data, x="KM Travelled", y="Profit/KM", hue="Company")
<AxesSubplot:xlabel='KM Travelled', ylabel='Profit/KM'>
c:\Users\rayng\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)

As was predicted, the distribution of the profit made per KM in a trip is about the same. Therefore, the next analysis we are interested in, is how many trips have earned a profit in each range of profits, for each company? We will make a histogram to plot the profit per KM.

Pink Cab:

cab_data[cab_data["Company"] == "Pink Cab"]["Profit/KM"].hist(bins=[5 * n - 5 for n in range(9)])
<AxesSubplot:>


##Probability of net profit/net loss per trip

cab_data["Loss"] = cab_data["Profit Generated"].map(lambda x: 1 if x < 0 else 0)
cab_data.groupby(["Company"])["Loss"].sum() / cab_data.groupby(["Company"])["Loss"].count()

# two models using the formula for each cab company:

Pink Cab:

Yellow Cab:

cab_data["Company2"] = cab_data["Company"].map(lambda x: 0 if x == "Pink Cab" else 1)
cab_data["Interaction"] = cab_data["KM Travelled"] * cab_data["Company2"]

reg = linreg(fit_intercept=True)
reg.fit(cab_data.dropna()[["Company2", "KM Travelled", "Interaction"]], cab_data.dropna()["Profit Generated"])

print(reg.intercept_)
print(reg.coef_)
-0.6944430460706883
## Customer demographics

customer_data["Age Group"] = customer_data["Age"].map(lambda x: (x//10) * 10)
customer_data["Income Bracket"] = customer_data["Income (USD/Month)"].map(lambda x: min((x//2000),20) * 2000)

pd.reset_option("all")
customer_data

Gender	Age	Income (USD/Month)	Age Group	Income Bracket
Customer ID					


trip_counts_pink = cab_data[cab_data["Company"] == "Pink Cab"].groupby(["Customer ID"]).count()["Trips"]
trip_counts_yellow = cab_data[cab_data["Company"] == "Yellow Cab"].groupby(["Customer ID"]).count()["Trips"]
trip_counts_pink
Customer ID
1        4
2        4
3        6
4        1
5        8
        ..
59995    2
59997    2
59998    3
59999    2
60000    4
Name: Trips, Length: 32330, dtype: int64
trip_counts_yellow
Customer ID
1        25
2        36
3        40
4        25
5        23
         ..
59996     4
59997     8
59998     6
59999     6
60000    14
Name: Trips, Length: 39896, dtype: int64
customer_data["Pink Cab Trips"] = trip_counts_pink
customer_data["Yellow Cab Trips"] = trip_counts_yellow

customer_data = customer_data.fillna(0)

customer_data
Gender	Age	Income (USD/Month)	Age Group	Income Bracket	Pink Cab Trips	Yellow Cab Trips
Customer ID							
29290	Male	28	10813	20	10000	1.0	2.0
27703	Male	27	9237	20	8000	2.0	2.0
28712	Male	53	11242	50	10000	1.0	3.0
28020	Male	23	23327	20	22000	3.0	1.0
27182	Male	33	8536	30	8000	1.0	3.0
...	...	...	...	...	...	...	...
12490	Male	33	18713	30	18000	0.0	1.0
14971	Male	30	15346	30	14000	0.0	0.0
41414	Male	38	3960	30	2000	0.0	1.0
41677	Male	23	19454	20	18000	0.0	1.0
39761	Female	32	10128	30	10000	0.0	1.0
49171 rows × 7 columns

customer_data.groupby("Income Bracket")["Pink Cab Trips", "Yellow Cab Trips"].mean()
C:\Users\rayng\AppData\Local\Temp\ipykernel_7956\3280618199.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  customer_data.groupby("Income Bracket")["Pink Cab Trips", "Yellow Cab Trips"].mean()
Pink Cab Trips	Yellow Cab Trips
Income Bracket		
2000	1.718916	5.512028
4000	1.695586	5.375788
6000	1.690899	5.484744
8000	1.719808	5.616220
10000	1.708824	5.719519
12000	1.749238	5.833333
14000	1.730739	5.521660
16000	1.718226	5.761021
18000	1.738462	5.529596
20000	1.736004	5.493837
22000	1.749674	5.638708
24000	1.687425	5.560878
26000	1.680493	5.699552
28000	1.817045	5.487500
30000	1.760186	5.814901
32000	1.771762	5.161359
34000	1.597046	5.839662